# Other tasks

In [1]:
from sympy import Symbol, factorial, diff, integrate, exp, sqrt, oo, solve
import numpy as np

from project1.iterations import *
from project1.classes import State

In [2]:
a = Symbol('a', positive=True)
b = Symbol('b', positive=True)
r = Symbol('r', positive=True)
r1 = Symbol('r1', positive=True)
r2 = Symbol('r2', positive=True)
Z = Symbol('Z', positive=True)
aaa = Symbol('aaa', positive=True)
bbb = Symbol('bbb', positive=True)

In [3]:
f1s = State(1,0)
f2p = State(2,1)
f3d = State(3,2)
f2s = State(2,0)

In [4]:
def optimize_ab(N, E0, a, b, aa=1, bb=1/2):
    for i in range(N):
        dEa = diff(E0.subs(Z,2), a)
        dEb = diff(E0.subs(Z,2), b)

        dEaa = diff(dEa, a).subs(a,aa).subs(b,bb)
        dEab = diff(dEa, b).subs(a,aa).subs(b,bb)
        dEba = diff(dEb, a).subs(a,aa).subs(b,bb)
        dEbb = diff(dEb, b).subs(a,aa).subs(b,bb)

        ans = solve((dEa.subs(a,aa).subs(b,bb) + (aaa - aa)*dEaa + (bbb - bb)*dEab, dEb.subs(a,aa).subs(b,bb) + (aaa - aa)*dEba + (bbb - bb) * dEbb), aaa, bbb)
        aa = ans[aaa]
        bb = ans[bbb]
        EX = (E0.subs(a,aa).subs(b,bb) * Z**2).subs(Z, 2)
        print("   Iter. step {}: a = {:.6f} , b = {:.6f} , E = {:.9f}".format(i, float(aa), float(bb), float(EX)))

    return float(EX)

In [5]:
def h1(bra, ket, variable=r, Z=2, l=0):
    prefactor = 1
    
    h_diff = -1/2 * diff(ket, variable, 2) - 1/variable * diff(ket, variable, 1) 
    h = l*(l+1)/(2*variable**2) - 1/variable
    
    integral = integrate(variable**2 * bra * (h_diff + h * ket), (variable, 0, oo))
    
    return integral.doit() * prefactor

In [6]:
def h2(bra1, bra2, ket1, ket2, l, var1 = r1, var2=r2, Z=1):    
    prefactor = 1/(2*l + 1)
    
    # branch r1 > r2
    integral1 = integrate(var1**2 * var2**2 * (bra1 * bra2) * var2**l * var1**(-l-1) * (ket1 * ket2), (var2, 0, var1), (var1, 0, oo))
    
    # branch r1 < r2
    integral2 = integrate(var1**2 * var2**2 * (bra1 * bra2) * var1**l * var2**(-l-1) * (ket1 * ket2), (var2, var1, oo), (var1, 0, oo))
    
    return prefactor * (integral1 + integral2)

## state 1S

In [7]:
e1 = h1(f1s(r, a), f1s(r, a), variable=r, Z=2, l=0)

## via relation 5.36
# E1sZ2 = 2 * e0 + 1/Z * (4*a**3)**2 * integral_I(2*a, 2*a, 2,2,0)

## via direct integration
E1sZ2 = 2 * e1 + 1/Z * h2(f1s(r1,a),f1s(r2,a),f1s(r1,a),f1s(r2,a), 0, var1=r1, var2=r2).simplify()

alpha_opt = solve(diff(E1sZ2, a).subs(Z,2),a)[0]
E1S = (E1sZ2.subs(a,alpha_opt) * Z**2).subs(Z,2)

print("Energy of He state 1S is: {:.6f}".format(float(E1S)))
print("Value from the book:  E = {}".format(-2.8476))

Energy of He state 1S is: -2.847656
Value from the book:  E = -2.8476


## state 2P

In [8]:
l1 = 0
l2 = 1

e0 = h1(f1s(r, a), f1s(r, a), variable=r, Z=2, l=l1)
e1 = h1(f2p(r, b), f2p(r, b), variable=r, Z=2, l=l2)

# hc = h2(f1s(r1,a),f2p(r2,b),f1s(r1,a),f2p(r2,b), l1, var1=r1, var2=r2, Z=2).simplify()
# he = 1/3 * h2(f1s(r1,a),f2p(r2,b),f1s(r2,a),f2p(r1,b), l2, var1=r1, var2=r2, Z=2).simplify()

hc = a**3 * b**5 / 6 * integral_I(2*a, b, 2, 4, l1)
he = a**3 * b**5 / 6 * integral_I(a+b/2, a+b/2, 3, 3, l2) / (2*l2 + 1)

for mult, sign in zip(['singlet','triplet'], [1, -1]):
    E2pZ2 = e0 + e1 + 1/Z * (hc + sign * he)
    opt = optimize_ab(3, E2pZ2, a,b)
    print("Energy of He {} state 2P is: {:.6f}\n".format(mult, opt))
    
print("\nValues from the book are:")
print("   singlet ... a = {} , b = {} , E = {}".format(1.0015,0.4823,-2.12239))
print("   triplet ... a = {} , b = {} , E = {}".format(0.9955,0.5445,-2.13069))

   Iter. step 0: a = 1.001508 , b = 0.482368 , E = -2.122390091
   Iter. step 1: a = 1.001512 , b = 0.482363 , E = -2.122390091
   Iter. step 2: a = 1.001512 , b = 0.482363 , E = -2.122390091
Energy of He singlet state 2P is: -2.122390

   Iter. step 0: a = 0.995702 , b = 0.544107 , E = -2.130691227
   Iter. step 1: a = 0.995593 , b = 0.544575 , E = -2.130691334
   Iter. step 2: a = 0.995593 , b = 0.544575 , E = -2.130691334
Energy of He triplet state 2P is: -2.130691


Values from the book are:
   singlet ... a = 1.0015 , b = 0.4823 , E = -2.12239
   triplet ... a = 0.9955 , b = 0.5445 , E = -2.13069


## state 3D

In [9]:
l1 = 0
l2 = 2

e0 = h1(f1s(r, a), f1s(r, a), variable=r, Z=2, l=l1)
e1 = h1(f3d(r, b), f3d(r, b), variable=r, Z=2, l=l2)

# hc = h2(f1s(r1,a),f3d(r2,b),f1s(r1,a),f3d(r2,b), l1, var1=r1, var2=r2, Z=2).simplify()
# he = 1/(2*l2+1) * h2(f1s(r1,a),f3d(r2,b),f1s(r2,a),f3d(r1,b), l2, var1=r1, var2=r2, Z=2).simplify()

hc = 4 * a**3 * (2*b/3)**7 / factorial(6) * integral_I(2*a, 2*b/3, 2, 6, l1)
he = 4 * a**3 * (2*b/3)**7 / factorial(6) * integral_I(a+b/3, a+b/3, 4, 4, l2) / (2*l2 + 1)

for mult, sign in zip(['singlet','triplet'], [1, -1]):
    E3dZ2 = e0 + e1 + 1/Z * (hc + sign * he)
    print("Energy of He {} state 3D is: {:.7f}".format(mult,optimize_ab(3, E3dZ2, a,b)))
    
print("\nValues from the book are:")
print("   singlet ... a = {} , b = {} , E = {}".format(1.0,0.4998,-2.05555))
print("   triplet ... a = {} , b = {} , E = {}".format(1.0,0.5004,-2.05557))

   Iter. step 0: a = 1.000011 , b = 0.499762 , E = -2.055546095
   Iter. step 1: a = 1.000011 , b = 0.499762 , E = -2.055546095
   Iter. step 2: a = 1.000011 , b = 0.499762 , E = -2.055546095
Energy of He singlet state 3D is: -2.0555461
   Iter. step 0: a = 0.999981 , b = 0.500424 , E = -2.055571815
   Iter. step 1: a = 0.999981 , b = 0.500424 , E = -2.055571815
   Iter. step 2: a = 0.999981 , b = 0.500424 , E = -2.055571815
Energy of He triplet state 3D is: -2.0555718

Values from the book are:
   singlet ... a = 1.0 , b = 0.4998 , E = -2.05555
   triplet ... a = 1.0 , b = 0.5004 , E = -2.05557


## triplet state 2S

In [10]:
l1 = 0
l2 = 0

e0 = h1(f1s(r, a), f1s(r, a), variable=r, Z=2, l=l1)
e1 = h1(f2s(r, b), f2s(r, b), variable=r, Z=2, l=l2)

# hc = h2(f1s(r1,a),f2s(r2,b),f1s(r1,a),f2s(r2,b), l1, var1=r1, var2=r2, Z=2)
# he = 1/(2*l2+1) * h2(f1s(r1,a),f2s(r2,b),f1s(r2,a),f2s(r1,b), l2, var1=r1, var2=r2, Z=2)

pre1s = 4 * a**3 
pre2s_1 = b**3 / 2
pre2s_2 = (- b**4 / 4)
pre2s_3 = (b**5 / 8 )

hc1 = pre1s * pre2s_1 * integral_I(2*a, b, 2, 2, l1)
hc2 = pre1s * pre2s_2 * integral_I(2*a, b, 2, 3, l1)
hc3 = pre1s * pre2s_3 * integral_I(2*a, b, 2, 4, l1)
hc = hc1 + 2*hc2 + hc3

he1 = pre1s * pre2s_1 * integral_I(a+b/2, a+b/2, 2, 2, l2) / (2*l2 + 1)
he2 = pre1s * pre2s_2 * integral_I(a+b/2, a+b/2, 2, 3, l2) / (2*l2 + 1)
he3 = pre1s * pre2s_3 * integral_I(a+b/2, a+b/2, 3, 3, l2) / (2*l2 + 1)
he = he1 + 2*he2 + he3

for mult, sign in zip(['triplet',], [-1,]):
    E2sZ2 = e0 + e1 + 1/Z * (hc + sign * he)
    print("Energy of He {} state 3D is: {:.6f}".format(mult, optimize_ab(3, E2sZ2, a,b)))

print("\nFor a = {} , b = {} , E = {:.6f}".format(1,1,(E2sZ2 * Z**2).subs(a,1).subs(b,1).subs(Z,2)))
print("For a = {} , b = {} , E = {:.6f}".format(1,0.5,(E2sZ2 * Z**2).subs(a,1).subs(b,0.5).subs(Z,2)))

print("\nValues from the book are:")
print("   triplet ... a = {} , b = {} , E = {}".format(1.0, 1.0, -2.124))
print("   triplet ... a = {} , b = {} , E = {}".format(1.0, 0.5, -2.157))
print("   triplet ... a = {} , b = {} , E = {}".format(1.0042, 0.6945, -2.17195))

   Iter. step 0: a = 1.005504 , b = 0.607140 , E = -2.208851109
   Iter. step 1: a = 1.005664 , b = 0.610139 , E = -2.208857079
   Iter. step 2: a = 1.005664 , b = 0.610144 , E = -2.208857079
Energy of He triplet state 3D is: -2.208857

For a = 1 , b = 1 , E = -2.124143
For a = 1 , b = 0.5 , E = -2.200587

Values from the book are:
   triplet ... a = 1.0 , b = 1.0 , E = -2.124
   triplet ... a = 1.0 , b = 0.5 , E = -2.157
   triplet ... a = 1.0042 , b = 0.6945 , E = -2.17195
